<a href="https://colab.research.google.com/github/varshachawan/PySparkApplication/blob/master/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q https://mirror.olnevhost.net/pub/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
!ls /usr/lib/jvm
!pwd

In [0]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-2.4.5-bin-hadoop2.7"


In [0]:
!echo $JAVA_HOME

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
spark.conf.set("spark.executor.memory","4g")
spark.conf.set("spark.driver.memory","4g")
spark.conf.set("spark.memory.fraction","0.9")


In [0]:
# !pip install -U pyarrow

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
import sys, urllib,tempfile

In [0]:
Base_Dir = '/tmp'
OUTPUT_FILE = os.path.join(Base_Dir,"credit_data.csv")

In [0]:
#credit_data = urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data",OUTPUT_FILE)

In [0]:
credit_df = spark.read.option("inferSchema","true").csv("LoanStats3a.csv", header=True)
# spark.read.format("csv").option("").load

In [0]:
credit_df.show()

In [0]:
credit_df.count()

28142

In [0]:
#cre_df = spark.read.format("csv").option("inferSchema","true").option("header","false")\
                                  .option("sep",",").load("/tmp/credit_data.csv")

In [0]:
display(credit_df)

In [0]:
credit_df.printSchema()

In [0]:
credit_df.createOrReplaceTempView("loan")

In [0]:


sqldf= spark.sql("select * from loan")

In [0]:
sqldf.show(10)

In [0]:
credit_df.describe().show()

In [0]:
spark.sql("select distinct emp_length from loan limit 10").show()

In [0]:
from pyspark.sql.functions import regexp_extract , regexp_replace , col


In [0]:
credit_df.printSchema()

In [0]:
df_sel=credit_df.select("term","home_ownership","grade","purpose","int_rate","loan_status","addr_state","application_type","loan_amnt","emp_length","annual_inc","dti","delinq_2yrs","revol_util","total_acc","num_tl_90g_dpd_24m","dti_joint")

In [0]:
df_sel.cache()

In [0]:
reg = 'years|year|\\+|\\<'

In [0]:
df_sel.select(regexp_replace(col("emp_length"),reg,"").alias("emp_length_clean"),col("emp_length")).show(2)

In [0]:
reg1 = '\\d+'

In [0]:
df_sel.select(regexp_extract(col("emp_length"),reg1,0)).show(2)

In [0]:
df_sel = df_sel.withColumn("term_clean",regexp_replace(col("term"),"months","")).withColumn("emp_cleaned",regexp_extract(col("emp_length"),"\\d+",0))

In [0]:
df_sel.stat.cov("annual_inc","loan_amnt")

In [0]:
df_sel.stat.corr("annual_inc","loan_amnt")

In [0]:
df_sel.createOrReplaceTempView("sel_table")

In [0]:
spark.sql("select corr(term_clean,loan_amnt) from sel_table").show()

In [0]:
df_sel.stat.crosstab("loan_status","grade").show()

In [0]:
freq= df_sel.stat.freqItems(['purpose','grade'],0.3)

In [0]:
freq.collect()

In [0]:
df_sel.groupBy('purpose').count().orderBy('count').show()


In [0]:
from pyspark.sql.functions import mean,stddev_pop,min,max,avg,count

In [0]:
from pyspark.sql.functions import isnan,when,count,col

In [0]:
qp =[0.25,0.5,0.75,0.9]
er=0.0
df_sel.stat.approxQuantile("loan_amnt",qp,er)

In [0]:
df_sel.na.drop("all",subset=["loan_status"])

In [0]:
df_sel.select([count(when(isnan(c)| col(c).isNull(),c)).alias(c) for c in df_sel.columns]).show()

In [0]:
df_sel=df_sel.withColumn("rev_cleaned",regexp_extract(col("revol_util"),"\\d+",0))
df_sel.select("rev_cleaned").show()

In [0]:
def fill_avg(df,colname):
  return df_sel.select(colname).agg(avg(colname))

In [0]:
from pyspark.sql.functions import lit,coalesce

In [0]:
rev_avg = fill_avg(df_sel,"rev_cleaned").first()[0]



df_sel = df_sel.withColumn("rev_avg",lit(rev_avg))

In [0]:
df_sel.select("rev_avg").show()

In [0]:
df_sel =df_sel.withColumn("rev_cleaned",coalesce(col("rev_cleaned"),col("rev_avg")))

In [0]:
df_sel= df_sel.withColumn("rev_cleaned",df_sel["rev_cleaned"].cast("double"))

In [0]:
df_sel.select("rev_cleaned").show(10)

In [0]:
# df_sel.filter(col(bad_loan)=='Yes').show()

In [0]:
df_sel.show(10,False)